https://github.com/Turoad/lanedet

## Setup

In [ ]:
!git clone https://github.com/Turoad/lanedet.git

Cloning into 'lanedet'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 499 (delta 25), reused 24 (delta 24), pack-reused 467
Receiving objects: 100% (499/499), 376.91 KiB | 19.84 MiB/s, done.
Resolving deltas: 100% (260/260), done.


In [ ]:
# !gdown --id "16tcs5RLf7TY4rJVtHKIC_3VlTOgMl5Qq"
# !gdown --id "1BJY4uComQsxNZcZU-Cr-D9Bqn-6Z2ybr"
# !gdown --id "1NnuONFed5PpKul6MqUWDoJ4g_6LdT7tz"
# !gdown --id "1n1hKiX7pR9YBWfp7doQXKaPnVtVfh9W0"
!gdown --id "1Ssxo3DZkrKiJEpHb744RrC-daAvir6Oa"

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gdown/download.py:32: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs4.BeautifulSoup(line, features="html.parser")
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1Ssxo3DZkrKiJEpHb744RrC-daAvir6Oa

but Gdown can't. Please check connections and permissions.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r drive/MyDrive/dataset dataset

In [ ]:
!unzip driver_182_30frame.zip -d CULane_dataset
!unzip driver_37_30frame.zip -d CULane_dataset
!unzip driver_161_90frame.zip -d CULane_dataset
!unzip laneseg_label_w16.zip -d CULane_dataset
!unzip list.zip -d CULane_dataset

In [ ]:
%cd /content/lanedet
!mkdir -p data
!ln -s CULane_dataset data/CULane

/content/lanedet


Install some particular library

In [ ]:
!pip install scikit-image
!pip install scikit-learn

In [ ]:
# Reading the contents of the existing requirements.txt file
file_path = 'requirements.txt'

# Creating the new requirements list
new_requirements = """pandas
addict
opencv-python
pytorch_warmup
tqdm
p_tqdm
imgaug>=0.4.0
Shapely==1.7.0
ujson==1.35
yapf
albumentations==0.4.6
mmcv==1.2.5
pathspec"""

# Writing the new requirements to the file
with open(file_path, 'w') as file:
    file.write(new_requirements)

In [ ]:
!pip install -r requirements.txt

In [ ]:
!python setup.py build develop

/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:85: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running build
running build_py
copying lanedet/__init__.py -> build/lib.linux-x86_64-cpython-310/lanedet
copying lanedet/datasets/tusimple.py -> build/lib.linux-x86_64-cpython-310/lanedet/datasets
copying lanedet/datasets/registry.py -> build/lib.linux-x86_64-cpython-310/lanedet/datasets
copying lanedet/datasets/base_dataset.py -> build/lib.linux-x86_64-cpython-310/lanedet/datasets
copying lanedet/datasets/culane.py -> build/lib.linux-x86_64-cpython-310/lanedet/datasets
copying lane

**If having request restart kernal, we should restart after the cell run**

## Run detect

**Replace code in file to fix bugs**

In [ ]:
import os

def replace_in_file(file_path, old_str1, new_str1, old_str2='###', new_str2='###'):
  with open(file_path, 'r') as file:
      config_content = file.readlines()

  new_cut_height = new_str1
  new_sample_y = new_str2

  # Replace old_str -> new_str
  for i, line in enumerate(config_content):
      if old_str1 in line:
          config_content[i] = new_cut_height
      if old_str2 in line: ##chưa đổi
          config_content[i] = new_sample_y
  # Rewrite new text in file
  with open(file_path, 'w') as file:
      file.writelines(config_content)

#Replace in resnet18_culane.py
config_path = '/content/lanedet/configs/ufld/resnet18_culane.py'
# replace_in_file(config_path, 'cut_height=0', 'cut_height=ori_img_h//4\n', 'sample_y = range(589, 230, -20)','sample_y = range(ori_img_h-1, cut_height, -15)\n')
replace_in_file(config_path, "dataset_path = './data/CULane'", "dataset_path = '/content/dataset'\n")

#Replace in mobilenet.py
file_path = '/content/lanedet/lanedet/models/backbones/mobilenet.py'
replace_in_file(file_path, 'from torchvision.models.utils import load_state_dict_from_url', 'from torch.hub import load_state_dict_from_url\n')

#Replace in transforms.py
file_path = '/content/lanedet/lanedet/datasets/process/transforms.py'
replace_in_file(file_path, 'assert (isinstance(size, collections.Iterable) and len(size) == 2)', '        assert (isinstance(size, collections.abc.Iterable) and len(size) == 2)\n') #thiếu chuyên nghiệp....


In [ ]:
!python main.py configs/ufld/resnet18_culane.py --gpus 0

2024-07-30 03:15:02,385 - lanedet.utils.recorder - INFO - Config: 
net = dict(
    type='Detector',
)

backbone = dict(
    type='ResNetWrapper',
    resnet='resnet18',
    pretrained=True,
    replace_stride_with_dilation=[False, False, False],
    out_conv=False,
)
featuremap_out_channel = 512

griding_num = 200
num_classes = 4
heads = dict(type='LaneCls',
        dim = (griding_num + 1, 18, num_classes))

optimizer = dict(
  type='SGD',
  lr=0.015,
  weight_decay=1e-4,
  momentum=0.9
)

epochs = 500
batch_size = 32
total_iter = (88880 // batch_size + 1) * epochs
import math
scheduler = dict(
    type = 'LambdaLR',
    lr_lambda = lambda _iter : math.pow(1 - _iter/total_iter, 0.9)
)


img_norm = dict(
    mean=[103.939, 116.779, 123.68],
    std=[1., 1., 1.]
)

ori_img_h = 590 
ori_img_w = 1640 
img_h = 288
img_w = 800
cut_height=0
sample_y = range(589, 230, -20)

train_process = [
    dict(type='RandomRotation', degree=(-6, 6)),
    dict(type='RandomUDoffsetLABEL', max_offset=100),


/content/lanedet


In [ ]:
import numpy as np
import torch
import cv2
import os
import os.path as osp
import glob
import argparse
from lanedet.datasets.process import Process
from lanedet.models.registry import build_net
from lanedet.utils.config import Config
from lanedet.utils.visualization import imshow_lanes
from lanedet.utils.net_utils import load_network
from pathlib import Path
from tqdm import tqdm

class Detect(object):
    def __init__(self, cfg):
        self.cfg = cfg
        self.processes = Process(cfg.val_process, cfg)
        self.net = build_net(self.cfg)
        self.net = torch.nn.parallel.DataParallel(
                self.net, device_ids = range(1)).cuda()
        self.net.eval()
        load_network(self.net, self.cfg.load_from)

    def preprocess(self, img_path):
        ori_img = cv2.imread(img_path)
        h, w = ori_img.shape[0], ori_img.shape[1]
        ori_img = cv2.resize(ori_img,(1640,590))
        img = ori_img[self.cfg.cut_height:, :, :].astype(np.float32)
        data = {'img': img, 'lanes': []}
        data = self.processes(data)
        data['img'] = data['img'].unsqueeze(0)
        data.update({'img_path':img_path, 'ori_img':ori_img, 'h':h, 'w':w})
        return data

    def inference(self, data):
        with torch.no_grad():
            data = self.net(data)
            data = self.net.module.get_lanes(data)
        return data

    def show(self, data):
        out_file = self.cfg.savedir
        if out_file:
            out_file = osp.join(out_file, osp.basename(data['img_path']))
        lanes = [lane.to_array(self.cfg) for lane in data['lanes']]
        imshow_lanes(data['ori_img'], lanes, show=self.cfg.show)
        img = data['ori_img']
        h = data['h']
        w = data['w']
        img = cv2.resize(img, (w,h))
        # data.update({'ori_img':img})
        if out_file:
            cv2.imwrite(out_file, img)

    def run(self, data):
        data = self.preprocess(data)
        data['lanes'] = self.inference(data)[0]
        if self.cfg.show or self.cfg.savedir:
            self.show(data)
        return data

def get_img_paths(path):
    p = str(Path(path).absolute())  # os-agnostic absolute path
    if '*' in p:
        paths = sorted(glob.glob(p, recursive=True))  # glob
    elif os.path.isdir(p):
        paths = sorted(glob.glob(os.path.join(p, '*.*')))  # dir
    elif os.path.isfile(p):
        paths = [p]  # files
    else:
        raise Exception(f'ERROR: {p} does not exist')
    return paths

def process(config, show, output_dir, input_dir, model):
    cfg = Config.fromfile(config)
    cfg.show = show
    cfg.savedir = output_dir
    cfg.load_from = model
    detect = Detect(cfg)
    paths = get_img_paths(input_dir)
    for p in tqdm(paths):
        detect.run(p)

if __name__ == '__main__':
    ##Params in process function
    os.makedirs('./vis', exist_ok=True)
    config = '/content/lanedet/configs/ufld/resnet18_culane.py'
    show = False
    output_dir = './vis'
    input_dir = '/content/dataset-test'
    model = '/content/lanedet/work_dirs/CULane/20240730_031502_lr_1e-02_b_32/ckpt/best.pth'
    ##Run
    process(config, show, output_dir, input_dir, model)


100%|██████████| 15/15 [00:00<00:00, 29.32it/s]


In [ ]:
!cd work_dirs//CULane/20240730_031502_lr_1e-02_b_32/ckpt

In [ ]:
!cp -r /content/lanedet/work_dirs/CULane/20240730_031502_lr_1e-02_b_32/ckpt/best.pth /content/drive/MyDrive